### Import libraries

In [9]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

from dataclasses import dataclass, field

import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Helper methods

In [2]:
# ------------------------ #
# Helper logging functions
# ------------------------ #
def print_log(text: str) -> None:
    """ Prints the log """
    print(f"[ log ]: {text}")

def print_error(text: str) -> None:
    """ Prints the error """
    print(f"[ error ]: {text}")
# -------------------------------------------------- #
# Helper functions for matrix related operations
# -------------------------------------------------- #
def graph_to_matrix(G: Union[nx.Graph, npmat.matrix]) -> npmat.matrix:
    """
    Converts a graph to a matrix
    """
    return nx.to_numpy_matrix(G) if isinstance(G, nx.Graph) else G

def matrix_to_graph(matrix: Union[nx.Graph, npmat.matrix]) -> nx.Graph:
    """
    Convert from a numpy matrix to a network graph
    """
    return nx.from_numpy_matrix(matrix) if isinstance(matrix, npmat.matrix) else matrix

def undirected_to_directed(graph: nx.Graph) -> nx.DiGraph:
    """
    Converts an undirected graph to a directed graph
    """
    di_graph = nx.DiGraph()
    di_graph.add_edges_from(graph.edges())
    return di_graph

def csv_to_matrix(csv_file: str) -> npmat.matrix:
    """
    Returns a matrix from a csv file
    """
    return npmat.asmatrix(pd.read_csv(csv_file, header=None, on_bad_lines="skip").to_numpy())

### Documentation

In [3]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, "data.csv")
docs_dir = join(parent_dir, 'docs')
if exists(docs_dir):
    doc_file = relpath(join(docs_dir, 'practical_works_linear_programing_v3.pdf'))
    IFrame(doc_file, width=1200, height=350)
matrix = csv_to_matrix(data_file)

#### Linear Programming via PuLP

In [4]:
# Create the variable to contain the problem data
problem = p.LpProblem(name="The Miracle Worker", sense=p.const.LpMaximize)

# Create problem variables
x = p.LpVariable(name="Medicine_1_units", lowBound=0, upBound=None, cat=p.LpInteger)
y = p.LpVariable(name="Medicine_2_units", lowBound=0, upBound=None, cat=p.LpInteger)

# The objective function is added to "problem" first
problem += 25*x + 20*y, "Health restored; to be maximized"

# The two contraints for the herbs
problem += 3*x + 4*y <= 25, "Herb A constraint"
problem += 2*x + y <= 10, "Herb B constraint"

# The problem data is written to an .lp file
problem.writeLP(filename=join(data_dir, "miracle_worker.lp"), writeSOS=1, mip=1, max_length=100)

# The problem is solved using PuLP's choice of solver
problem.solve()

/opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


[Medicine_1_units, Medicine_2_units]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/05450cfbed73439281d4bb1a6abbf724-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/05450cfbed73439281d4bb1a6abbf724-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 18 RHS
At line 21 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 155 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of -155 found by DiveCoefficient after 0 iterations and

1

#### Output

In [5]:
print_log(f"{p.LpStatus[problem.status] = }")

_ = [print_log(f"{v.name} = {v.varValue}") for v in problem.variables()]

print_log(f"{p.value(problem.objective) = }")

[ log ]: p.LpStatus[problem.status] = 'Optimal'
[ log ]: Medicine_1_units = 3.0
[ log ]: Medicine_2_units = 4.0
[ log ]: p.value(problem.objective) = 155.0


#### Toy example (Linear Programming via PuLP)

In [6]:
# Create the variable to contain the problem data
problem = p.LpProblem(name="Toy Manufacturing", sense=p.const.LpMaximize)

# Create problem variables
x = p.LpVariable(name="Toy_1_units", lowBound=0, upBound=None, cat=p.LpInteger)
y = p.LpVariable(name="Toy_2_units", lowBound=0, upBound=None, cat=p.LpInteger)

# The objective function is added to "problem" first
problem += 25*x + 20*y, "Profit; to be maximized"

# The two contraints for the herbs
problem += 20*x + 12*y <= 2000, "Required units - constraint"
problem += 5*x + 5*y <= 540, "Time required - constraint"

# The problem data is written to an .lp file
problem.writeLP(filename=join(data_dir, "toy_manufacturing.lp"), writeSOS=1, mip=1, max_length=100)

# The problem is solved using PuLP's choice of solver
problem.solve()

[Toy_1_units, Toy_2_units]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/mambaforge/base/envs/decision_modelling/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/25fbf009db8140fa98c88686e1393238-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/25fbf009db8140fa98c88686e1393238-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 18 RHS
At line 21 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2600 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of -2600 found by DiveCoefficient after 0 iterations a

1

#### Output

In [7]:
print_log(f"{p.LpStatus[problem.status] = }")

_ = [print_log(f"{v.name} = {v.varValue}") for v in problem.variables()]

print_log(f"{p.value(problem.objective) = }")

[ log ]: p.LpStatus[problem.status] = 'Optimal'
[ log ]: Toy_1_units = 88.0
[ log ]: Toy_2_units = 20.0
[ log ]: p.value(problem.objective) = 2600.0


### How to visit Paris ? (efficiently with low budget)

In [10]:
@dataclass(frozen=False, order=False)
class SiteInfo:
    """
    A dataclass to hold the site information
    """
    name: str = field(default="")
    site_code: str = field(default="")
    price: float = field(default=0.0)
    duration: float = field(default=0.0) # in hours
    rating: int = field(default=0)


TypeError: field() got an unexpected keyword argument 'describe'

In [ ]:
sites = ["TE", "ML", "AT", "MO", "JT", "CA", "CP", "CN", "BS", "SC", "PC", "TM", "AC"]

distance_in_kms = (
    list[0, 3.8, 2.1, 2.4, 3.5, 4.2, 5.0,  4.4, 5.5, 4.2, 2.5, 3.1, 1.9],
    list[0,   0, 3.8, 1.1, 1.3, 3.3, 1.3,  1.1, 3.4, 0.8, 1.7, 2.5, 2.8],
    list[0,   0,   0, 3.1, 3.0, 5.8, 4.8,  4.9, 4.3, 4.6, 2.2, 4.4, 1.0],
    list[0,   0,   0,   0, 0.9, 3.1, 2.5,  2.0, 3.9, 1.8, 1.0, 2.3, 2.1],
    list[0,   0,   0,   0,   0, 4.2, 2.0,  2.4, 2.7, 2.0, 1.0, 3.4, 2.1],
    list[0,   0,   0,   0,   0,   0, 3.5,  2.7, 6.5, 2.6, 3.8, 1.3, 4.9],
    list[0,   0,   0,   0,   0,   0,   0, 0.85, 3.7, 0.9, 2.7, 3.4, 3.8],
    list[0,   0,   0,   0,   0,   0,   0,    0, 4.5, 0.4, 2.8, 2.7, 3.9],
    list[0,   0,   0,   0,   0,   0,   0,    0,   0, 4.2, 3.3, 5.7, 3.8],
    list[0,   0,   0,   0,   0,   0,   0,    0,   0,   0, 2.5, 2.6, 3.6],
    list[0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0, 3.0, 1.2],
    list[0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0,   0, 2.1],
    list[0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0,   0,   0]
    )
a=np.matrix(matrix)
a = a.astype(float)
df=pd.DataFrame(np.matrix(a.transpose())+a)
df
# df